In [1]:
import matplotlib.pyplot as plt
%matplotlib inline

from lstm_chem.utils.config import process_config
from lstm_chem.model import LSTMChem
from lstm_chem.generator import LSTMChemGenerator

In [2]:
CONFIG_FILE = 'experiments/2019-12-19/LSTM_Chem/config.json'
config = process_config(CONFIG_FILE)

In [3]:
modeler = LSTMChem(config, session='generate')

Loading model architecture from experiments/2019-12-19/LSTM_Chem/model_arch.json ...
Loading model checkpoint from experiments/2019-12-19/LSTM_Chem/checkpoints/LSTM_Chem-22-0.04.hdf5 ...
Loaded the Model.


In [ ]:
# Wow, it took more than 3 days !
generator = LSTMChemGenerator(modeler)
sampled_smiles = generator.sample(num=30000)

  0%|          | 85/30000 [04:42<43:07:12,  5.19s/it] 

In [ ]:
import pickle
with open('./datasets/randomly_sampled_smiles.pkl', 'wb') as f:
    pickle.dump(sampled_smiles, f)

In [ ]:
from rdkit import RDLogger, Chem, DataStructs
from rdkit.Chem import AllChem, Draw
from rdkit.Chem.Draw import IPythonConsole
RDLogger.DisableLog('rdApp.*')

In [ ]:
mols = []
for smi in sampled_smiles:
    mol = Chem.MolFromSmiles(smi)
    if mol is not None:
        mols.append(mol)
print(len(mols))

In [ ]:
with open('./datasets/dataset_cleansed.smi') as f:
    org_smiles = [l.rstrip() for l in f]

In [ ]:
import numpy as np

In [ ]:
org_sampled_smiles = np.random.choice(org_smiles, size=10000, replace=False)

In [ ]:
org_mols = [Chem.MolFromSmiles(smi) for smi in org_sampled_smiles]